# Praktikum Pengantar Pembelajaran Mesin


---
## Bab 6. Support Vector Machine (SVM)


### 1) Import Data

Unduh dataset yang akan digunakan pada praktikum kali ini. Anda dapat menggunakan aplikasi wget untuk mendowload dataset dan menyimpannya dalam Google Colab. Jalankan cell di bawah ini untuk mengunduh dataset

In [1]:
! wget https://gist.githubusercontent.com/netj/8836201/raw/6f9306ad21398ea43cba4f7d537619d0e07d5ae3/iris.csv

--2024-04-21 03:55:46--  https://gist.githubusercontent.com/netj/8836201/raw/6f9306ad21398ea43cba4f7d537619d0e07d5ae3/iris.csv
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3975 (3.9K) [text/plain]
Saving to: ‘iris.csv’

iris.csv            100%[===================>]   3.88K  --.-KB/s    in 0s      

2024-04-21 03:55:46 (60.5 MB/s) - ‘iris.csv’ saved [3975/3975]



Setelah dataset berhasil diunduh, langkah berikutnya adalah membaca dataset dengan memanfaatkan fungsi **readcsv** dari library pandas. Lakukan pembacaan berkas csv ke dalam dataframe dengan nama **data** menggunakan fungsi **readcsv**. Jangan lupa untuk melakukan import library pandas terlebih dahulu


In [2]:
import pandas as pd
import numpy as np
data = pd.read_csv('iris.csv')



Cek isi dataset Anda dengan menggunakan perintah **head()**

In [3]:
data.head()

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa
2,4.7,3.2,1.3,0.2,Setosa
3,4.6,3.1,1.5,0.2,Setosa
4,5.0,3.6,1.4,0.2,Setosa


Metode Support Vector Machine (SVM) dasar hanya mampu melakukan klasifikasi data yang terdiri dari dua kelas, atau disebut klasifikasi biner. Dataset iris memiliki 3 kelas, sehingga salah satu kelasnya, yaitu Iris-virginica perlu dihapus. Cell berikut menghapus data yang memiliki kelas Iris-virginica

In [4]:
data.drop(data[data['variety']=='Virginica'].index,inplace=True)

SVM juga memiliki keterbatasan, yaitu hanya mampu menerima kelas dalam bentuk numerik. Cell berikut mengubah kelas menjadi bilangan, Iris setosa menjadi -1 dan Iris-versicolor menjadi 1

In [5]:
data['variety'] = data['variety'].map({'Setosa':-1, 'Versicolor':1})

Tampilkan beberapa data untuk mengecek hasilnya

In [6]:
data.head()

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,-1
1,4.9,3.0,1.4,0.2,-1
2,4.7,3.2,1.3,0.2,-1
3,4.6,3.1,1.5,0.2,-1
4,5.0,3.6,1.4,0.2,-1


## 2) Membagi data menjadi data latih dan data uji

Metode pembelajaran mesin memerlukan dua jenis data :


1.   Data latih : Digunakan untuk proses training metode klasifikasi
2.   Data uji : Digunakan untuk proses evaluasi metode klasifikasi

Data uji dan data latih perlu dibuat terpisah (mutualy exclusive) agar hasil evaluasi lebih akurat.

Data uji dan data latih dapat dibuat dengan cara membagi dataset dengan rasio tertentu, misalnya 80% data latih dan 20% data uji.

Library Scikit-learn memiliki fungsi [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) pada modul **model_selection** untuk membagi dataset menjadi data latih dan data uji. Bagilah dataset anda menjadi dua, yaitu **data_latih** dan **data_uji**.


In [7]:
from sklearn.model_selection import train_test_split
data_latih, data_uji = train_test_split(data, test_size = 0.2)

Tampilkan banyaknya data pada **data_latih** dan **data_uji**. Seharusnya **data_latih** terdiri dari 80 data, dan **data_uji** terdiri dari 20 data

In [8]:
print(data_latih.shape[0])
print(data_uji.shape[0])

80
20


Pisahkan label/kategori dari data latih dan data uji menjadi variabel tersendiri. Beri nama variabelnya  **label_latih** dan **label_uji**

In [9]:
label_latih = data_latih.pop('variety')
label_uji = data_uji.pop('variety')

In [12]:
label_latih.head()

47   -1
66    1
16   -1
60    1
86    1
Name: variety, dtype: int64

## 3) Proses Training

Tujuan dari algoritma SVM adalah meminimalkan nilai *cost function*. Penghitungan nilai minimal dapat dapat dilakukan dengan menghitung nilai gradien dari *cost function* terlebih dahulu. Fungsi di bawah ini berguna untuk menghitung nilai gradien cost function

In [14]:
def hitung_cost_gradient(W, X, Y, regularization):
  jarak = 1 - (Y * np.dot(X, W))
  dw = np.zeros(len(W))
  if max(0, jarak) == 0:
    di = W
  else:
    di = W - (regularization * Y * X)
  dw += di
  return dw

Terdapat beberapa cara untuk meminimalkan nilai *cost function*, salah satunya menggunakan Stochastic Gradient Descent (SGD) untuk melakukan minimasi. Minimasi *cost function* merupakan inti dari algoritma SVM. Fungsi di bawah ini merupakan implementasi algoritma SGD

In [15]:
from sklearn.utils import shuffle
def sgd(data_latih, label_latih, learning_rate=0.000001, max_epoch=1000, regularization=10000):
  data_latih = data_latih.to_numpy()
  label_latih = label_latih.to_numpy()
  bobot = np.zeros(data_latih.shape[1])
  for epoch in range(1, max_epoch):
    X, Y = shuffle(data_latih, label_latih, random_state=101)
    for index, x in enumerate(X):
      delta = hitung_cost_gradient(bobot, x, Y[index], regularization)
      bobot = bobot - (learning_rate * delta)
  return bobot

Proses training dilakukan dengan memanggil fungsi **sgd** dengan parameter input berupa data latih dan label latih. Parameter learning rate dan max epoch menggunakan nilai *default* nya

In [16]:
W = sgd(data_latih, label_latih)
print(W)

[-0.26588053 -0.64163893  1.15727661  0.52748864]


## 4) Proses *testing*
Proses testing dilakukan dengan menghitung nilai [*dot product*](https://en.wikipedia.org/wiki/Dot_product) antara bobot hasil training dengan data uji. Kelas data ditentukan berdasarkan tanda (positif atau negatif) dari hasil dot product tersebut. Fungsi berikut mengimplementasikan proses testing

In [17]:
def testing(W, data_uji):
  prediksi = np.array([])
  for i in range(data_uji.shape[0]):
    y_prediksi = np.sign(np.dot(W, data_uji.to_numpy()[i]))
    prediksi = np.append(prediksi, y_prediksi)
  return prediksi

Lakukan pengujian menggunakan seluruh data uji. Bandingkan hasilnya dengan nilai label sebenarnya untuk menghitung berapa banyak data uji yang berhasil diprediksi dengan benar.

In [18]:
y_prediksi = testing(W, data_uji)
print(sum(y_prediksi==label_uji))

20


## TUGAS
Pada tugas kali ini Anda masih melakukan klasifikasi dataset iris, tetapi menggunakan kelas Iris-versicolor dan Iris-virginica. Apakah hasilnya lebih baik jika dibandingkan dengan Iris-setosa dan Iris-versicolor?

In [19]:
dataTugas = pd.read_csv('iris.csv')

In [20]:
dataTugas.head()

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa
2,4.7,3.2,1.3,0.2,Setosa
3,4.6,3.1,1.5,0.2,Setosa
4,5.0,3.6,1.4,0.2,Setosa


In [21]:
dataTugas.drop(dataTugas[dataTugas['variety']=='Setosa'].index,inplace=True)

In [22]:
dataTugas['variety'] = dataTugas['variety'].map({'Virginica':-1, 'Versicolor':1})

In [23]:
tugas_latih, tugas_uji = train_test_split(dataTugas, test_size = 0.2, random_state = 101)

In [24]:
print(tugas_uji.shape[0])
print(tugas_latih.shape[0])

20
80


In [25]:
label_tlatih = tugas_latih.pop('variety')
label_tuji = tugas_uji.pop('variety')

In [26]:
label_tlatih.head()

111   -1
97     1
104   -1
107   -1
52     1
Name: variety, dtype: int64

In [27]:
W = sgd(tugas_latih, label_tlatih)
print(W)

[ 3.71010489  3.65497888 -4.81431102 -6.85848154]


In [28]:
y_prediksi = testing(W, tugas_uji)
print(sum(y_prediksi==label_tuji))

19


Hasilnya tidak lebih baik karena hanya mampu mengenali 17 diantara 20 data uji.

Lakukan proses training ulang, tetapi menggunakan nilai max_epoch sebesar 5000. Anda dapat memberikan parameter max_epoch=5000 saat memanggil fungsi **sgd**. Apakah hasilnya lebih baik? Mengapa?

In [29]:
from sklearn.utils import shuffle
def sgd_2(data_latih, label_latih, learning_rate=0.000001, max_epoch=5000, regularization=10000):
  data_latih = data_latih.to_numpy()
  label_latih = label_latih.to_numpy()
  bobot = np.zeros(data_latih.shape[1])
  for epoch in range(1, max_epoch):
    X, Y = shuffle(data_latih, label_latih, random_state=101)
    for index, x in enumerate(X):
      delta = hitung_cost_gradient(bobot, x, Y[index], regularization)
      bobot = bobot - (learning_rate * delta)
  return bobot

In [30]:
W = sgd_2(tugas_latih, label_tlatih)
print(W)

[ 4.93533064  3.58949584 -5.74400006 -8.33755769]


In [31]:
y_prediksi = testing(W, tugas_uji)
print(sum(y_prediksi==label_tuji))

20


Berdasarkan hasil percobaan yang telah dilakukan, ditemukan ketika nilai max_epoch=5000 digunakan, tingkat akurasi mencapai 100% dari 20 data uji. Hal ini disebabkan oleh peningkatan jumlah iterasi yang dilakukan, sehingga gradient yang dihasilkan menjadi lebih bervariasi untuk menentukan hasil klasifikasi.

Terakhir, lakukanlah seleksi fitur sederahana dengan mengambil hanya fitur sepal length dan sepal width saja. Apakah hasilnya lebih baik?

In [32]:
tugas2_latih = tugas_latih.copy()
tugas2_uji = tugas_uji.copy()

In [33]:
tugas2_latih.head()

,sepal.length,sepal.width,petal.length,petal.width
111,6.4,2.7,5.3,1.9
97,6.2,2.9,4.3,1.3
104,6.5,3.0,5.8,2.2
107,7.3,2.9,6.3,1.8
52,6.9,3.1,4.9,1.5


In [34]:
tugas2_uji.head()

,sepal.length,sepal.width,petal.length,petal.width
66,5.6,3.0,4.5,1.5
51,6.4,3.2,4.5,1.5
93,5.0,2.3,3.3,1.0
117,7.7,3.8,6.7,2.2
139,6.9,3.1,5.4,2.1


In [35]:
pop1 = tugas2_latih.pop('petal.length')
pop2 = tugas2_latih.pop('petal.width')
pop3 = tugas2_uji.pop('petal.length')
pop4 = tugas2_uji.pop('petal.width')

In [36]:
W = sgd_2(tugas2_latih, label_tlatih)
print(W)

[-0.37729705  1.12778453]


In [37]:
y_prediksi = testing(W, tugas2_uji)
print(sum(y_prediksi==label_tuji))

13


In [38]:
X = sgd(tugas2_latih, label_tlatih)
print(X)

[-0.38449547  1.12712834]


In [39]:
x_prediksi = testing(W, tugas2_uji)
print(sum(x_prediksi==label_tuji))

13


percobaan diatas menunjukkan bahwa tingkat akurasi turun menjadi hanya 65% atau 13 dari 20 data uji, meskipun kedua percobaan menggunakan nilai max_epoch yang sama yaitu 1000 dan 5000. Penyebabnya adalah jumlah fitur pada dataset yang diuji semakin berkurang. Semakin sedikit fitur yang digunakan, semakin sedikit pula informasi yang tersedia untuk melakukan seleksi spesies, yang kemudian berdampak pada tingkat ketelitian yang menurun. Oleh karena itu, hasil akhir dari percobaan ini jauh lebih rendah dibandingkan dengan tugas 1 dan 2. :)

-Ahmad Zaki 225150201111025-